In [1]:
NODE_COUNT = 4
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/RING/fedstellar_DFL_18_07_2024_14_17_34/metrics/participant_"
date_str = "18/07/2024 12:17:34	" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/RING/fedstellar_DFL_18_07_2024_14_17_34/metrics/participant_0
    step  value                     timestamp                    tag
0  13324   23.9 2024-07-18 12:27:13.631815168  Resources/CPU_percent
1  13334    5.4 2024-07-18 12:27:23.655913728  Resources/CPU_percent
2  13324    0.0 2024-07-18 12:27:13.631848448     Resources/CPU_temp
3  13334    0.0 2024-07-18 12:27:23.655951360     Resources/CPU_temp
4  13324   21.0 2024-07-18 12:27:13.631861760  Resources/RAM_percent
2024-07-18 12:33:40 2024-07-18 12:17:42
                 step     value                     timestamp
tag                                                          
Test/Accuracy   16569  0.869587 2024-07-18 12:33:31.209806336
Test/F1Score    16569  0.842060 2024-07-18 12:33:31.209837056
Test/Loss       16569  0.491258 2024-07-18 12:33:31.209773312
Test/Precision  16569  0.859741 2024-07-18 12:33:31.209817600
Test/Recall     16569  0.869587 2024-07-18 12:33:31.209827840
/

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,16569,0.869587,2024-07-18 12:33:31.209806336,participant_0
Test/F1Score,16569,0.842060,2024-07-18 12:33:31.209837056,participant_0
Test/Loss,16569,0.491258,2024-07-18 12:33:31.209773312,participant_0
Test/Precision,16569,0.859741,2024-07-18 12:33:31.209817600,participant_0
Test/Recall,16569,0.869587,2024-07-18 12:33:31.209827840,participant_0
Test/Accuracy,16572,0.920127,2024-07-18 12:33:30.853327872,participant_1
Test/F1Score,16572,0.907112,2024-07-18 12:33:30.853357568,participant_1
Test/Loss,16572,0.459870,2024-07-18 12:33:30.853296896,participant_1
Test/Precision,16572,0.919891,2024-07-18 12:33:30.853338624,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_40226/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,15183.25,0.8948
Test/F1Score,15183.25,0.8754
Test/Loss,15183.25,0.4780
Test/Precision,15183.25,0.8921
Test/Recall,15183.25,0.8948


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_40226/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,2775.5,0.03
Test/F1Score,2775.5,0.03
Test/Loss,2775.5,0.02
Test/Precision,2775.5,0.03
Test/Recall,2775.5,0.03


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_40226/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,15183.25,0.8948
Test/F1Score,15183.25,0.8754
Test/Loss,15183.25,0.4780
Test/Precision,15183.25,0.8921
Test/Recall,15183.25,0.8948
